# Transmission lines

PowerGenome compiles information on the transmission constraints between model regions and the centroid-to-centroid distance between model regions.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd
from powergenome.generators import load_ipm_shapefile
from powergenome.GenX import (
    network_line_loss,
    network_max_reinforcement,
    network_reinforcement_cost,
)
from powergenome.transmission import (
    agg_transmission_constraints,
    transmission_line_distance,
)
from powergenome.util import init_pudl_connection, load_settings

## Import settings

In [3]:
pudl_engine, pudl_out = init_pudl_connection()
cwd = Path.cwd()

settings_path = (
    cwd.parent / "example_system" / "test_settings.yml"
)
settings = load_settings(settings_path)

## Aggregate transmission constraints
EPA provides bulk transmission constraints between IPM regions. PowerGenome combines these constraints across aggregated model regions. The required settings parameters are:
- `model_regions`
- `region_aggregations`

In [4]:
settings["model_regions"]

['CA_N', 'CA_S', 'WECC_AZ']

In [5]:
settings["region_aggregations"]

{'CA_N': ['WEC_CALN', 'WEC_BANC'],
 'CA_S': ['WEC_LADW', 'WECC_SCE', 'WEC_SDGE', 'WECC_IID']}

In [6]:
transmission = agg_transmission_constraints(pudl_engine=pudl_engine, settings=settings)

In [7]:
transmission

,Network_lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,Transmission Path Name
0,1,1,-1,0,3675.0,-3000.0,CA_N_to_CA_S
1,2,0,1,-1,3767.0,-4794.0,CA_S_to_WECC_AZ


## Add centroid-to-centroid distance

In [8]:
model_regions_gdf = load_ipm_shapefile(settings)
model_regions_gdf

/Users/Home/opt/miniconda3/envs/powergenome/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


,IPM_Region,geometry,model_region
0,WEC_BANC,"MULTIPOLYGON (((-137062.964 -20577.766, -13352...",CA_N
1,WEC_CALN,"MULTIPOLYGON (((-26905.041 -343681.519, -26858...",CA_N
2,WEC_LADW,"MULTIPOLYGON (((147531.138 -578110.363, 147523...",CA_S
3,WEC_SDGE,"MULTIPOLYGON (((261508.595 -578272.714, 261412...",CA_S
4,WECC_AZ,"POLYGON ((971403.331 -56900.550, 1043399.918 -...",WECC_AZ
5,WECC_IID,"POLYGON ((499663.896 -494815.927, 497706.026 -...",CA_S
6,WECC_SCE,"MULTIPOLYGON (((43384.114 -525298.005, 50284.5...",CA_S


In [9]:
transmission_line_distance(
    trans_constraints_df=transmission,
    ipm_shapefile=model_regions_gdf,
    settings=settings,
)

,Network_lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,Transmission Path Name,distance_mile
0,1,1,-1,0,3675.0,-3000.0,CA_N_to_CA_S,336.6
1,2,0,1,-1,3767.0,-4794.0,CA_S_to_WECC_AZ,321.0


## Line loss, reinforcement costs, max reinforcement
These functions use the settings parameters:
- `transmission_investment_costs.tx.capex_mw_mile.<model_region>` (a nested dictionary with a single value for each region. Transmission between regions uses an average of the two values)
- `transmission_investment_costs.tx.wacc` (a numeric value for the weighted average cost of capital within the nested dictionary)
- `transmission_investment_costs.tx.investment_years` (a numeric value for the investment timeframe within the nested dictionary)
- `tx_line_loss_100_miles`
- `tx_expansion_per_period`

In [10]:
settings["transmission_investment_cost"]["tx"]

{'capex_mw_mile': {'CA_N': 3037.5, 'CA_S': 3037.5, 'WECC_AZ': 1350},
 'wacc': 0.069,
 'investment_years': 60}

In [11]:
settings["tx_line_loss_100_miles"]

0.01

In [12]:
settings["tx_expansion_per_period"]

1.0

In [13]:
network_line_loss(transmission=transmission, settings=settings)

,Network_lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,Transmission Path Name,distance_mile,Line_Loss_Percentage
0,1,1,-1,0,3675.0,-3000.0,CA_N_to_CA_S,336.6,0.0337
1,2,0,1,-1,3767.0,-4794.0,CA_S_to_WECC_AZ,321.0,0.0321


In [14]:
network_reinforcement_cost(transmission=transmission, settings=settings)

,Network_lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,Transmission Path Name,distance_mile,Line_Loss_Percentage,Line_Reinforcement_Cost_per_MW_yr
0,1,1,-1,0,3675.0,-3000.0,CA_N_to_CA_S,336.6,0.0337,74220.0
1,2,0,1,-1,3767.0,-4794.0,CA_S_to_WECC_AZ,321.0,0.0321,51119.0


In [15]:
network_max_reinforcement(transmission=transmission, settings=settings)

,Network_lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,Transmission Path Name,distance_mile,Line_Loss_Percentage,Line_Reinforcement_Cost_per_MW_yr,Line_Max_Reinforcement_MW
0,1,1,-1,0,3675.0,-3000.0,CA_N_to_CA_S,336.6,0.0337,74220.0,3675.0
1,2,0,1,-1,3767.0,-4794.0,CA_S_to_WECC_AZ,321.0,0.0321,51119.0,3767.0
